# 試み
###  sentence-BERT => Decoder の連結でどうにかならんか
- 心配なのはベクトル

In [1]:
import sys
sys.path.append("../")
from datatools.analyzer import *

/home/yamada/.local/share/virtualenvs/twitchAI-AEfI_JA6/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from sentence_transformers import SentenceTransformer
# bert_path = "../../corpus/pretrained/sbert_coherence"
# sbert = SentenceTransformer(bert_path)

In [3]:
from transformer_base import *

In [4]:
out_path = "../../corpus/novel_formated/"
# corpus_name = "novel_segments.tsv"
corpus_name = "ntt_segment_plain2.tsv"

import csv
conv_data = []
with open(out_path+corpus_name, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    conv_data = [row for row in reader]

In [5]:
lim=1000
import random
random.seed(0)
conv_data_mini = random.sample(conv_data, lim)

In [6]:
def make_Src_Tgt(conv_data):
    src_str = []
    tgt_str = []

    for conv in conv_data:
        src_str.append(conv[0])
        tgt_str.append(conv[1])
    return src_str, tgt_str

src_str, tgt_str = make_Src_Tgt(conv_data_mini)

In [7]:
from collections import Counter
from torchtext.vocab import Vocab

def build_vocab(texts, tokenizer):
    
    counter = Counter()
    for text in tqdm(texts):
        counter.update(tokenizer(text))
    return Vocab(counter, specials=['<unk>', '<pad>','<fos>', '<eos>', '<del>'])

In [8]:
src_trainval_str, src_test_str, tgt_trainval_str, tgt_test_str= train_test_split(src_str, tgt_str, test_size=0.20, random_state=5)
print("len=> src_train_val:{0}, src_test:{1}".format(len(src_trainval_str), len(src_test_str)))

src_train_str, src_val_str, tgt_train_str, tgt_val_str= train_test_split(src_trainval_str, tgt_trainval_str, test_size=0.10, random_state=5)

print("len=> src_train:{0}, src_val:{1}".format(len(src_train_str), len(src_val_str)))

src_train_str_ = src_train_str[:lim]
tgt_train_str_ = tgt_train_str[:lim]
src_val_str_ = src_val_str[:lim//10]
tgt_val_str_ = tgt_val_str[:lim//10]

len=> src_train_val:800, src_test:200
len=> src_train:720, src_val:80


In [9]:
tokenizer_src = mecab_tokenize
tokenizer_tgt = mecab_tokenize

vocab_src = build_vocab(src_train_str_, tokenizer=tokenizer_src)
vocab_tgt = build_vocab(tgt_train_str_, tokenizer=tokenizer_tgt)

100%|██████████| 720/720 [00:00<00:00, 31318.63it/s]


In [10]:
# from sentence_transformers import SentenceTransformer
# from sentence_transformers import models

# download_path = "../../corpus/"
# # download_path = ""
# transformer = models.Transformer(download_path+'cl-tohoku/bert-base-japanese-whole-word-masking')
# pooling = models.Pooling(transformer.get_word_embedding_dimension(),    
#   pooling_mode_mean_tokens=True,
#   pooling_mode_cls_token=False, 
#   pooling_mode_max_tokens=False
# )
# model = SentenceTransformer(modules=[transformer, pooling])

# sentences = ['吾輩は猫である',  '本日は晴天なり']
# embeddings = model.encode(sentences)

# for i, embedding in enumerate(embeddings):
#   print("[%d] : %s" % (i, embedding.shape, ))

from sentence_transformers import SentenceTransformer
# from sentence_transformers import models

# bert_path = "../../corpus/pretrained/sbert_unclear1"
bert_path = "../../corpus/pretrained/sbert_coherence"
sbert = SentenceTransformer(bert_path)

[595] 2022-06-17 19:27:06,394 Info sentence_transformers.SentenceTransformer :Load pretrained SentenceTransformer: ../../corpus/pretrained/sbert_coherence
[595] 2022-06-17 19:27:07,200 Info sentence_transformers.SentenceTransformer :Use pytorch device: cuda


In [11]:
def data_process(texts_src, texts_tgt, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt):
    
    data = []
    src_vec = sbert.encode(texts_src)
    for (src, tgt) in zip(src_vec, texts_tgt):
        # src_tensor = torch.tensor(
        #     convert_text_to_indexes(text=src, vocab=vocab_src, tokenizer=tokenizer_src),
        #     dtype=torch.long
        # )
        tgt_tensor = torch.tensor(
            convert_text_to_indexes(text=tgt, vocab=vocab_tgt, tokenizer=tokenizer_tgt),
            dtype=torch.long
        )
        data.append((torch.tensor(src) , tgt_tensor))
        
    return data

def convert_text_to_indexes(text, vocab, tokenizer):
    return [vocab['<fos>']] + [
        vocab[token] for token in tokenizer(text.strip("\n"))
    ] + [vocab['<eos>']]

In [12]:
train_data = data_process(
    texts_src=src_train_str_, texts_tgt=tgt_train_str_,
    vocab_src=vocab_src, vocab_tgt=vocab_tgt,
    tokenizer_src=tokenizer_src, tokenizer_tgt=tokenizer_tgt
)
valid_data = data_process(
    texts_src=src_val_str_, texts_tgt=tgt_val_str_,
    vocab_src=vocab_src, vocab_tgt=vocab_tgt,
    tokenizer_src=tokenizer_src, tokenizer_tgt=tokenizer_tgt
)

Batches: 100%|██████████| 3/3 [00:00<00:00, 12.09it/s]


In [22]:
batch_size = 32
PAD_IDX = vocab_src['<pad>']
START_IDX = vocab_src['<fos>']
END_IDX = vocab_src['<eos>']

In [23]:
def generate_batch(data_batch):
    
    batch_src, batch_tgt = [], []
    for src, tgt in data_batch:
        batch_src.append(src)
        batch_tgt.append(tgt)
        
    # batch_src = pad_sequence(batch_src, padding_value=PAD_IDX, batch_first=True)
    # batch_tgt = pad_sequence(batch_tgt, padding_value=PAD_IDX, batch_first=True)
    batch_src = pad_sequence(batch_src, padding_value=PAD_IDX, batch_first=False)
    batch_tgt = pad_sequence(batch_tgt, padding_value=PAD_IDX, batch_first=False)
    
    return batch_src, batch_tgt

In [24]:
train_iter = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(valid_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)

In [13]:
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import (
    TransformerEncoder, TransformerDecoder,
    TransformerEncoderLayer, TransformerDecoderLayer
)
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

import math
class TokenEmbedding(nn.Module):
    
    def __init__(self, vocab_size, embedding_size):
        
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=PAD_IDX)
        self.embedding_size = embedding_size
        
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.embedding_size)

class DecoderForBERT(nn.Module):
    
    # 予めembeddingしたsrcを入力
    def __init__(
        self,
        num_decoder_layers: int,
        embedding_size: int, vocab_size_tgt: int, 
        dim_feedforward:int = 512, dropout:float = 0.1, nhead:int = 8
        ):

        self.positional_encoding = PositionalEncoding(embedding_size, dropout=dropout)
        self.token_embedding_tgt = TokenEmbedding(vocab_size_tgt, embedding_size)
        # デコーダーを定義
        decoder_layer = TransformerDecoderLayer(
            d_model=embedding_size, nhead=nhead, dim_feedforward=dim_feedforward
        )
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.output = nn.Linear(embedding_size, vocab_size_tgt)
    
    def forward(
        self, src: Tensor, tgt: Tensor,
        mask_tgt: Tensor, padding_mask_tgt: Tensor,
        memory_key_padding_mask: Tensor
        ):

        embedding_tgt = self.positional_encoding(self.token_embedding_tgt(tgt))

        outs = self.transformer_decoder(
            embedding_tgt, src, mask_tgt, None,
            padding_mask_tgt, memory_key_padding_mask
        )

        return self.output(outs)



In [27]:
def create_mask_tgt(tgt, PAD_IDX):
    
    seq_len_tgt = tgt.shape[0]

    mask_tgt = generate_square_subsequent_mask(seq_len_tgt)

    padding_mask_tgt = (tgt == PAD_IDX).transpose(0, 1)
    
    return mask_tgt, padding_mask_tgt

def train(model, data, optimizer, criterion, PAD_IDX):
    
    model.train()
    losses = 0
    for src, tgt in tqdm(data):
        
        src = src.to(device)
        tgt = tgt.to(device)

        input_tgt = tgt[:-1, :]

        # mask_src, mask_tgt, padding_mask_src, padding_mask_tgt = create_mask(src, input_tgt, PAD_IDX)
        mask_tgt, padding_mask_tgt = create_mask_tgt(tgt, PAD_IDX)
        # print(src.shape, tgt.shape)

        logits = model(
            src=src, tgt=input_tgt,
            mask_tgt=mask_tgt, padding_mask_tgt=padding_mask_tgt,
            memory_key_padding_mask=padding_mask_src
        )

        optimizer.zero_grad()

        output_tgt = tgt[1:, :]
        loss = criterion(logits.reshape(-1, logits.shape[-1]), output_tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

        del src
        del tgt
        del logits
        del loss
        
    return losses / len(data)


def evaluate(model, data, criterion, PAD_IDX):
    
    model.eval()
    losses = 0
    for src, tgt in data:
        
        src = src.to(device)
        tgt = tgt.to(device)

        input_tgt = tgt[:-1, :]

        mask_src, mask_tgt, padding_mask_src, padding_mask_tgt = create_mask(src, input_tgt, PAD_IDX)

        logits = model(
            src=src, tgt=input_tgt,
            mask_src=mask_src, mask_tgt=mask_tgt,
            padding_mask_src=padding_mask_src, padding_mask_tgt=padding_mask_tgt,
            memory_key_padding_mask=padding_mask_src
        )
        
        output_tgt = tgt[1:, :]
        loss = criterion(logits.reshape(-1, logits.shape[-1]), output_tgt.reshape(-1))
        losses += loss.item()
        
    return losses / len(data)

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]], device='cuda:0')

In [ ]:
vocab_size_src = len(vocab_src)
vocab_size_tgt = len(vocab_tgt)
embedding_size = 768
nhead = 8
dim_feedforward = 768
num_encoder_layers = 2
num_decoder_layers = 4
dropout = 0.1

model = DecoderForBERT(
    num_decoder_layers=num_decoder_layers,
    embedding_size=embedding_size,
    vocab_size_src=vocab_size_src, vocab_size_tgt=vocab_size_tgt,
    dim_feedforward=dim_feedforward,
    dropout=dropout, nhead=nhead
)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
import time

epoch = 30
best_loss = float('Inf')
best_model = None
patience = 10
counter = 0

for loop in range(1, epoch + 1):
    
    start_time = time.time()
    
    loss_train = train(
        model=model, data=train_iter, optimizer=optimizer,
        criterion=criterion, PAD_IDX=PAD_IDX
    )
    
    elapsed_time = time.time() - start_time
    
    loss_valid = evaluate(
        model=model, data=valid_iter, criterion=criterion, PAD_IDX=PAD_IDX
    )
    
    print('[{}/{}] train loss: {:.2f}, valid loss: {:.2f}  [{}{:.0f}s] count: {}, {}'.format(
        loop, epoch,
        loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        counter,
        '**' if best_loss > loss_valid else ''
    ))
    
    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = model
        counter = 0
        
    if counter > patience:
        break
    
    counter += 1